## Домашка 4

In [1]:
import pandas as pd

In [3]:
train, test = pd.read_csv('train_6b.csv'), pd. read_csv('test_6b.csv')
# с диплома остался парафразер с очень тщательной нормализацией
print(train.head())
print(train.shape)
print(test.head())
print(test.shape)

   Unnamed: 0                                      предложение 1  \
0           0  Полицейским разрешат стрелять на поражение по ...   
1           1  Право полицейских на проникновение в жилище ре...   
2           2  Президент Египта ввел чрезвычайное положение в...   
3           3  Вернувшихся из Сирии россиян волнует вопрос тр...   
4           4  В Москву из Сирии вернулись 2 самолета МЧС с р...   

                                       предложение 2  класс  \
0  Полиции могут разрешить стрелять по хулиганам ...      0   
1  Правила внесудебного проникновения полицейских...      0   
2  Власти Египта угрожают ввести в стране чрезвыч...      0   
3  Самолеты МЧС вывезут россиян из разрушенной Си...     -1   
4  Самолеты МЧС вывезут россиян из разрушенной Си...      0   

                                             леммы 1  \
0  полицейский разрешать стрелять поражение гражд...   
1  право полицейский проникновение жилище решать ...   
2  президент египет вводить чрезвычайный пол

In [4]:
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.ensemble import RandomForestClassifier
import gensim

In [7]:
import csv
import os

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

In [29]:
# чтение одного мистического корпуса
sentences = []
sentences_raw = []
for file in os.listdir(os.path.join(os.getcwd(),'corpus')):
    with open('corpus/' + file, 'r', encoding='utf-8') as  f:
        sentence = []
        reader = csv.reader(f, delimiter=';')
        for line in reader:
            word = line[2]
            if word.isalpha():
                sentence.append(word.lower())
        sentences.append(sentence)
        sentences_raw.append(' '.join(sentence))

In [30]:
print(sentences[0][:20])

['год', 'на', 'ямал', 'более', 'или', 'менее', 'нормальный', 'погода', 'держаться', 'в', 'сочи', 'несколько', 'день', 'начинать', 'портиться', 'связанный', 'это', 'с', 'прохождение', 'атмосферный']


In [37]:
cv = CountVectorizer( max_features=1000)
X = cv.fit_transform(sentences_raw)

In [40]:
svd = TruncatedSVD(50)
svd.fit(X)
id2word = {i:w for i,w in enumerate(cv.get_feature_names())}
word2id = {w:i for i,w in id2word.items()}
id2vec_svd = svd.components_.T
def most_similar(word, id2vec):
    similar = [id2word[i] for i in cosine_distances(id2vec[word2id[word]].reshape(1, -1), id2vec).argsort()[0][:10]]
    return similar
print(most_similar('погода', id2vec_svd))

['погода', 'мчс', 'снежный', 'пожарный', 'столица', 'госпитализировать', 'температура', 'чп', 'сообщение', 'встречный']
